In [ ]:
from webdriver_manager.chrome import ChromeDriverManager
import csv
import os
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium import webdriver
from urllib.parse import urljoin
import requests
import os
import traceback
from icecream import ic
import base64
import datetime
import csv
import os
from selenium.webdriver.common.by import By
import pandas as pd
import locale


def remove_claim_button(driver):
    remove_script = """
    var element = document.evaluate(arguments[0], document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue;
    if (element) {
        element.parentNode.removeChild(element);
    }
    """

    # XPath of the element to be removed
    xpath_to_remove = '//*[@id="teamfresh"]/div/div/div[1]'

    # Execute the script
    driver.execute_script(remove_script, xpath_to_remove)


# get_all_sms 함수화
#
def get_all_sms(driver):
    extract_sms = True
    try:
        # Click on the first element
        first_element_xpath = (
            '//*[@id="teamfresh"]/div/div/div[2]/div[2]/div/table[1]/tbody/tr/td[2]/div'
        )
        first_element = driver.find_element(By.XPATH, first_element_xpath)
        first_element.click()
        time.sleep(0.5)
    except NoSuchElementException:
        traceback.print_exc()
        print("Element not found. SMS 추출 취소")
        extract_sms = False

    if extract_sms:
        # Wait for the new element to appear and get its text
        new_element_xpath = "/html/body/div[2]/div/div[1]/div/div/div[2]/p"
        new_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, new_element_xpath))
        )
        ic(new_element)
        ic(new_element.text)
        tmp_text = driver.execute_script("return arguments[0].innerHTML;", new_element)
        ic(tmp_text)
        new_text = new_element.text

        # close the popup
        close_xpath = "/html/body/div[2]/div/div[1]/div/div/div[1]/button"
        close_bttn = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, close_xpath))
        )
        close_bttn.click()

        # Set the new text to the target element
        target_element_xpath = (
            '//*[@id="teamfresh"]/div/div/div[2]/div[2]/div/table[1]/tbody/tr/td[2]'
        )
        target_element = driver.find_element(By.XPATH, target_element_xpath)
        driver.execute_script(
            "arguments[0].innerText = arguments[1];", target_element, new_text
        )


#
# get_image 함수화
#
def get_image(driver):
    current_url = driver.current_url
    try:
        # Check and click the specified element
        icon_xpath = (
            '//*[@id="teamfresh"]/div/div/div[2]/div[2]/div/table[1]/tbody/tr/td[3]/i'
            if "shipping" in current_url
            else '//*[@id="teamfresh"]/div/div/div/div[2]/div/table[1]/tbody/tr/td[3]/i'
        )
        icon_element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, icon_xpath))
        )
        icon_element.click()
        # Wait for the new image element and get its 'src' attribute
        img_xpath = '//*[@id="imgTag"]'
        img_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, img_xpath))
        )
        blob_img_url = img_element.get_attribute("src")
        image_bytes = get_file_content_chrome(driver, blob_img_url)

        # Download the image and save it locally
        img_folder = "img"
        os.makedirs(img_folder, exist_ok=True)
        img_name = blob_img_url.split("/")[-1] + ".jpg"
        img_path = os.path.join(img_folder, img_name)
        with open(img_path, "wb") as img_file:
            img_file.write(image_bytes)

        time.sleep(1)
        # Click the button to return to the previous screen
        back_button_xpath = (
            "/html/body/div[2]/div/div[1]/div/div/div[1]/button"
            if "shipping" in current_url
            else "/html/body/div[2]/div/div[1]/div/div/div[1]/h5/div/button"
        )
        back_button = driver.find_element(By.XPATH, back_button_xpath)
        back_button.click()
        time.sleep(0.5)

        # Replace the original icon element with the downloaded image
        driver.execute_script(
            """
            var iconElement = arguments[0];
            var newImg = document.createElement('img');
            newImg.src = arguments[1];
            iconElement.parentNode.replaceChild(newImg, iconElement);
            """,
            icon_element,
            img_path,
        )
        time.sleep(0.5)
    except NoSuchElementException:
        traceback.print_exc()
        print("Element not found. 이미지 추출 취소")
        # ESC 키를 보내어 팝업을 무시
        WebDriverWait(driver, 2).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        ).send_keys(Keys.ESCAPE)


def select_second_tab(driver):
    # 두 번째 탭으로 이동
    element_to_click = driver.find_element(
        By.XPATH, '//*[@id="teamfresh"]/div/div/ul/li[2]/a'
    )
    element_to_click.click()


def embed_js_for_tabselection(driver):
    # 페이지의 HTML 내용 가져오기
    html_content = driver.page_source

    # 탭선택 가능하게 하는 JavaScript snippet
    javascript_snippet = """
    <script>
    document.addEventListener('DOMContentLoaded', function() {
        var tabs = document.querySelectorAll('.nav-item .nav-link');
        var tabPanes = document.querySelectorAll('.tab-content .tab-pane');
        tabs.forEach(function(tab, index) {
            tab.addEventListener('click', function(event) {
                event.preventDefault();
                tabs.forEach(t => t.classList.remove('active'));
                tabPanes.forEach(pane => pane.classList.remove('active'));
                tab.classList.add('active');
                tabPanes[index].classList.add('active');
            });
        });
    });
    </script>
    """
    # Insert the JavaScript snippet before the closing </body> tag
    html_content = html_content.replace("</body>", javascript_snippet + "</body>")
    return html_content


def get_css_and_link_to_local(driver, html_content):
    # 'css' 및 'js' 폴더 생성
    css_folder = "css"
    os.makedirs(css_folder, exist_ok=True)

    # 기본 경로 설정
    base_path = "./"  # 웹 서버에서의 경로

    # CSS 및 JavaScript 파일 처리
    for tag in ["link"]:
        elements = driver.find_elements(By.TAG_NAME, tag)
        for element in elements:
            href = (
                element.get_attribute("href")
                if tag == "link"
                else element.get_attribute("src")
            )

            if href and (
                (tag == "link" and "stylesheet" in element.get_attribute("rel"))
                or tag == "script"
            ):
                # 절대 URL로 변환
                # old_href = href
                # ic(old_href)

                href = urljoin(remote_url, href)
                file_name = href.split("/")[-1]
                file_path = os.path.join(css_folder, file_name)

                # 파일 다운로드 및 저장
                response = requests.get(href)
                with open(file_path, "wb") as file:
                    file.write(response.content)

                # HTML 내용에서 링크 경로 수정
                new_href = base_path + os.path.join(css_folder, file_name)
                old_href = "/static/" + os.path.join(css_folder, file_name)
                # ic(old_href)
                # ic(new_href)
                html_content = html_content.replace(old_href, new_href)
    return html_content


def get_file_content_chrome(driver, uri):
    result = driver.execute_async_script(
        """
        var uri = arguments[0];
        var callback = arguments[1];
        var xhr = new XMLHttpRequest();
        xhr.responseType = 'arraybuffer';
        xhr.onload = function() {
            var reader = new FileReader();
            reader.readAsDataURL(new Blob([xhr.response]));
            reader.onloadend = function() {
                callback(reader.result);
            }
        };
        xhr.onerror = function() { callback(xhr.status) };
        xhr.open('GET', uri);
        xhr.send();
    """,
        uri,
    )
    if isinstance(result, int):
        raise Exception("Request failed with status %s" % result)
    return base64.b64decode(result.split(",")[1])


def navigate_to_page(url, wait_xpath, sec_to_wait=10):
    try:
        driver.get(url)
        WebDriverWait(driver, sec_to_wait).until(
            EC.presence_of_element_located((By.XPATH, wait_xpath))
        )
        time.sleep(sec_to_wait)
    except Exception as e:
        print(f"Error navigating to {url}: {e}")


def open_and_login(driver):
    # 페이지 로드
    driver.get(remote_url)
    time.sleep(1)

    # 로그인 페이지 확인 및 로그인 수행
    try:
        # ID와 Password 입력 필드를 찾아 값을 입력
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.ID, "userLoginId"))
        )
        id_field = driver.find_element(By.ID, "userLoginId")
        password_field = driver.find_element(By.ID, "userLoginPw")

        id_field.send_keys("dummyid")
        password_field.send_keys("dummypassword")

        # 로그인 버튼 클릭
        login_button = driver.find_element(
            By.XPATH, '//*[@id="teamfresh"]/div/div/form/button'
        )
        login_button.click()
    except Exception as e:
        print("로그인 페이지를 찾을 수 없습니다:", e)


# 한국어 환경을 설정합니다.
# 윈도우의 경우 locale.setlocale(locale.LC_TIME, 'Korean') 를 사용하세요.
locale.setlocale(locale.LC_TIME, "ko_KR.UTF-8")


def parse_korean_month_year(month_year_str):
    # "1월 2024" 형식의 문자열을 datetime.date 객체로 변환합니다.
    return datetime.datetime.strptime(month_year_str, "%B %Y").date()


def format_date_korean(date):
    # 날짜를 "1월 2024" 형식으로 변환합니다.
    return date.strftime("%B %Y")


def select_date(driver, target_date):
    # XPath selectors
    date_input_xpath = "//*[@id='teamfresh']/div[2]/div[2]/form/div/div/div[2]/div/label[3]/div/div[1]/div/input"
    current_month_year_xpath = "//*[@id='teamfresh']/div[2]/div[2]/form/div/div/div[2]/div/label[3]/div/div[2]/div[2]/div/div/div[2]/div[1]/div[1]"
    prev_month_button_xpath = "//*[@id='teamfresh']/div[2]/div[2]/form/div/div/div[2]/div/label[3]/div/div[2]/div[2]/div/div/button"
    next_month_button_xpath = "//*[@id='teamfresh']/div[2]/div[2]/form/div/div/div[2]/div/label[3]/div/div[2]/div[2]/div/div/button[2]"
    wait = WebDriverWait(driver, 5)
    # Open calendar
    driver.find_element(By.XPATH, date_input_xpath).click()
    time.sleep(0.5)  # Wait for calendar to open

    # Get the current displayed month and year from the calendar
    current_month_year_text = driver.find_element(
        By.XPATH, current_month_year_xpath
    ).text  # e.g., "1월 2024"

    # Parse the current month and year to a datetime.date object
    current_month_year = datetime.datetime.strptime(
        current_month_year_text, "%B %Y"
    ).date()

    cnt = 0
    # Navigate the calendar to the correct month and year
    while current_month_year != target_date.replace(day=1):
        print("looping", cnt)
        cnt += 1
        if current_month_year > target_date.replace(day=1):
            driver.find_element(By.XPATH, prev_month_button_xpath).click()
        else:
            driver.find_element(By.XPATH, next_month_button_xpath).click()
        time.sleep(0.3)  # Wait for calendar to update
        current_month_year_text = driver.find_element(
            By.XPATH, current_month_year_xpath
        ).text
        current_month_year = datetime.datetime.strptime(
            current_month_year_text, "%B %Y"
        ).date()

    # Click the correct day
    # Construct the XPath for the target date
    # Assuming the days in the calendar are zero-padded to 3 digits
    day_xpath = f"//div[contains(@class, 'react-datepicker__day') and not(contains(@class, 'react-datepicker__day--outside-month')) and text()='{target_date.day}']"
    # Wait for the target day to be visible and clickable
    wait.until(EC.visibility_of_element_located((By.XPATH, day_xpath)))
    wait.until(EC.element_to_be_clickable((By.XPATH, day_xpath)))

    # Click the correct day
    driver.find_element(By.XPATH, day_xpath).click()

    # Assuming that selecting the date will close the calendar
    time.sleep(1)  # Wait for calendar to close


def show_daily_data(driver, year, month, daty):
    select_date(driver, datetime.date(year, month, daty))

    # Click the search button
    search_button = driver.find_element(By.XPATH, "//*[@id='searchBtn']")
    search_button.click()


def get_all_table_data(driver):
    # Now proceed with the rest of the script
    # Extract table data and store it in a DataFrame
    table = driver.find_element(
        By.XPATH, "//*[@id='teamfresh']/div[2]/div[2]/div[2]/table"
    )
    rows = table.find_elements(By.TAG_NAME, "tr")
    data = []
    for row in rows:
        cols = row.find_elements(By.TAG_NAME, "td")
        data.append([col.text for col in cols])

    df = pd.DataFrame(
        data[1:], columns=[col.text for col in rows[0].find_elements(By.TAG_NAME, "th")]
    )
    df["tms상세주소"] = pd.Series(dtype="str")
    return df


def create_filename_from_url(url, save_root, *sub_folders):
    # 폴더 경로를 구성합니다. save_root 다음에 sub_folders 요소들을 추가합니다.
    folder_path = os.path.join(save_root, *sub_folders)
    # 폴더가 존재하지 않으면 생성합니다.
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    url_parts = url.split("/")
    a = url_parts[-3]  # 마지막에서 세 번째 요소
    b = url_parts[-1]  # 마지막 요소
    filename = f"{a}-{b}.html"
    return os.path.join(folder_path, filename)


def validate_and_extract_date_parts(date_str):
    try:
        # 날짜 객체를 생성합니다.
        date_obj = datetime.strptime(date_str, "%Y-%m-%d")

        # 연도, 월, 일을 추출합니다.
        year = date_obj.year
        month = date_obj.month
        day = date_obj.day

        # YYYYMM 형식의 문자열을 생성합니다.
        yyyymm_str = date_obj.strftime("%Y%m")

        return year, month, day, yyyymm_str
    except ValueError:
        # 입력된 날짜가 올바른 형식이 아닌 경우 예외 처리
        raise ValueError(
            f"Invalid date format: {date_str}. Expected format: YYYY-MM-DD."
        )


def save_dataframe_to_csv(df, filename, sub_folder):
    # 지정된 서브 폴더 경로를 생성합니다.
    save_path = os.path.join("./", sub_folder)

    # 폴더가 존재하지 않으면 생성합니다.
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    # 파일 전체 경로를 생성합니다.
    file_path = os.path.join(save_path, filename)

    # 파일이 존재하는 경우 데이터를 추가하고, 존재하지 않는 경우 새 파일을 생성합니다.
    if os.path.exists(file_path):
        df.to_csv(file_path, mode="a", header=False, index=False, encoding="utf-8-sig")
    else:
        df.to_csv(file_path, mode="w", index=False, encoding="utf-8-sig")

In [ ]:
def process_tcc_page(driver, tcc_page_url, yy, mm, dd, ym):
    navigate_to_page(
        tcc_page_url,
        '//*[@id="teamfresh"]/div[2]/div[2]/form/div/div/div[1]/div/div/label/div[1]',
        3,
    )

    show_daily_data(driver, yy, mm, dd)
    time.sleep(2)
    df = get_all_table_data(driver)

    if len(
        driver.find_elements(
            By.XPATH,
            "//*[@id='teamfresh']/div[2]/div[2]/div[2]/table/tbody/tr/td[21]/button",
        )
    ) != len(df):
        raise ValueError(
            "Mismatch between number of rows in DataFrame and number of detail buttons"
        )

    main_window_handle = driver.current_window_handle

    for i, button in enumerate(
        driver.find_elements(
            By.XPATH,
            "//*[@id='teamfresh']/div[2]/div[2]/div[2]/table/tbody/tr/td[21]/button",
        )
    ):
        button.click()
        time.sleep(1)

        for handle in driver.window_handles:
            if handle != main_window_handle:
                driver.switch_to.window(handle)
                break

        # 상세 페이지 처리 로직
        select_second_tab(driver)
        time.sleep(1)
        get_all_sms(driver)
        get_image(driver)
        if "shipping" in driver.current_url:
            remove_claim_button(driver)
        html_content = embed_js_for_tabselection(driver)
        html_content = get_css_and_link_to_local(driver, html_content)

        detail_html_file_path = create_filename_from_url(
            driver.current_url, "./tms", ym
        )
        with open(detail_html_file_path, "w", encoding="utf-8") as file:
            file.write(driver.page_source)

        df.at[i, "tms상세주소"] = driver.current_url
        df.at[i, "상세보기"] = detail_html_file_path

        driver.close()
        driver.switch_to.window(main_window_handle)

    return df


# 메인 코드 실행 부분
profile_path = "~/.config/google-chrome/Profile 9"
chrome_options = Options()
chrome_options.add_argument("--user-data-dir=" + profile_path)
chrome_service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=chrome_service, options=chrome_options)

remote_url = "https://tms.teamfresh.co.kr"
open_and_login(driver)
time.sleep(1)

# 날짜 관련 정보 설정
yy, mm, dd, ym = validate_and_extract_date_parts("2023-10-04")

# 첫 번째 URL 처리
tcc_page_url1 = "https://tms.teamfresh.co.kr/tcc/delivery/report"
df1 = process_tcc_page(driver, tcc_page_url1, yy, mm, dd, ym)

# 두 번째 URL 처리
tcc_page_url2 = "https://tms.teamfresh.co.kr/tcc/return/report"
df2 = process_tcc_page(driver, tcc_page_url2, yy, mm, dd, ym)

driver.quit()  # 브라우저를 닫습니다.


save_dataframe_to_csv(df1, "tms_shipping.csv", "shipping_data")
save_dataframe_to_csv(df2, "tmp_return.csv", "return_data")
# 결과 DataFrame
df1, df2

In [ ]:
# def navigate_to_page(url, wait_xpath, sec_to_wait=10):
#     try:
#         driver.get(url)
#         WebDriverWait(driver, sec_to_wait).until(
#             EC.presence_of_element_located((By.XPATH, wait_xpath))
#         )
#         time.sleep(sec_to_wait)
#     except Exception as e:
#         print(f"Error navigating to {url}: {e}")


# def open_and_login(driver):
#     # 페이지 로드
#     driver.get(remote_url)
#     time.sleep(1)

#     # 로그인 페이지 확인 및 로그인 수행
#     try:
#         # ID와 Password 입력 필드를 찾아 값을 입력
#         WebDriverWait(driver, 5).until(
#             EC.presence_of_element_located((By.ID, "userLoginId"))
#         )
#         id_field = driver.find_element(By.ID, "userLoginId")
#         password_field = driver.find_element(By.ID, "userLoginPw")

#         id_field.send_keys("dummyid")
#         password_field.send_keys("dummypassword")

#         # 로그인 버튼 클릭
#         login_button = driver.find_element(
#             By.XPATH, '//*[@id="teamfresh"]/div/div/form/button'
#         )
#         login_button.click()
#     except Exception as e:
#         print("로그인 페이지를 찾을 수 없습니다:", e)


profile_path = "~/.config/google-chrome/Profile 9"
chrome_options = Options()
chrome_options.add_argument("--user-data-dir=" + profile_path)
chrome_service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=chrome_service, options=chrome_options)

# 원격 페이지 URL
remote_url = "https://tms.teamfresh.co.kr"
open_and_login(driver)
time.sleep(1)


# shipping 배송 상세 페이지 로드
tcc_page_url = "https://tms.teamfresh.co.kr/tcc/delivery/report"
navigate_to_page(
    tcc_page_url,
    '//*[@id="teamfresh"]/div[2]/div[2]/form/div/div/div[1]/div/div/label/div[1]',
    3,
)
# # return 배송 상세 페이지 로드
# tcc_page_url = "https://tms.teamfresh.co.kr/tcc/return/report"
# navigate_to_page(
#     tcc_page_url,
#     '//*[@id="teamfresh"]/div[2]/div[2]/div[1]/div/div/button[4]',
#     3,
# )

yy, mm, dd, ym = validate_and_extract_date_parts("2023-10-04")

show_daily_data(driver, yy, mm, dd)
time.sleep(2)
df = get_all_table_data(driver)
# Check if the number of rows matches the number of buttons
buttons = driver.find_elements(
    By.XPATH, "//*[@id='teamfresh']/div[2]/div[2]/div[2]/table/tbody/tr/td[21]/button"
)
if len(buttons) != len(df):
    raise ValueError(
        "Mismatch between number of rows in DataFrame and number of detail buttons"
    )

# 현재 창의 핸들을 저장합니다.
main_window_handle = driver.current_window_handle

# 각 "상세보기" 버튼을 클릭하고 상세 페이지를 처리합니다.
for i, button in enumerate(buttons):
    button.click()
    time.sleep(1)  # 새 창이 열리기를 기다립니다.

    # 새 창으로 전환합니다.
    for handle in driver.window_handles:
        if handle != main_window_handle:
            driver.switch_to.window(handle)
            break

    # 상세 페이지 처리 로직
    select_second_tab(driver)
    time.sleep(1)
    get_all_sms(driver)
    get_image(driver)
    if "shipping" in driver.current_url:
        remove_claim_button(driver)
    html_content = embed_js_for_tabselection(driver)
    html_content = get_css_and_link_to_local(driver, html_content)

    # 수정된 HTML 내용을 파일로 저장합니다.
    detail_html_file_path = create_filename_from_url(driver.current_url, "./tms", ym)
    with open(detail_html_file_path, "w", encoding="utf-8") as file:
        file.write(html_content)

    # URL 및 파일 경로를 DataFrame에 저장합니다.
    df.at[i, "tms상세주소"] = driver.current_url
    df.at[i, "상세보기"] = detail_html_file_path

    # 새 창을 닫고 메인 창으로 돌아갑니다.
    driver.close()
    driver.switch_to.window(main_window_handle)


driver.quit()  # 브라우저를 닫습니다.

df
# The DataFrame 'df' now contains the table data